In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
from IPython.display import display, Markdown
import snakemd
import pandas as pd

from garmindb import ConfigManager, Graph
from garmindb.garmindb import GarminSummaryDb, DaysSummary, MonitoringDb, MonitoringHeartRate
from garmindb.summarydb import DaysSummary, WeeksSummary, MonthsSummary, SummaryDb

from jupyter_funcs import format_number

# Plots how RHR changes over the years

In [ ]:
period = ConfigManager.graphs_activity_config("hr", 'days')

In [ ]:
start_ts = datetime.datetime.combine(datetime.date(year=2017, month=1, day=1), datetime.datetime.min.time())
end_ts = datetime.datetime.combine(datetime.date.today(), datetime.datetime.max.time())

db_params = ConfigManager.get_db_params()
sum_db = SummaryDb(db_params, False)
data = DaysSummary.get_for_period(sum_db, start_ts, end_ts, DaysSummary)
time = [entry.day for entry in data]
hr_avg = [entry.hr_avg for entry in data]
hr_min = [entry.hr_min for entry in data]
hr_max = [entry.hr_max for entry in data]
rhr_avg = [entry.rhr_avg for entry in data]
rhr_min = [entry.rhr_min for entry in data]
rhr_max = [entry.rhr_max for entry in data]
inactive_hr_avg = [entry.inactive_hr_avg for entry in data]
inactive_hr_min = [entry.inactive_hr_min for entry in data]
inactive_hr_max = [entry.inactive_hr_max for entry in data]

hr_df = pd.DataFrame([time, hr_avg, hr_min, hr_max, rhr_avg, rhr_min, rhr_max,
                      inactive_hr_avg, inactive_hr_min, inactive_hr_max]).T
hr_df.columns = ["Date", "hr_avg", "hr_min", "hr_max", "rhr_avg", "rhr_min", "rhr_max",
                 "inactive_hr_avg", "inactive_hr_min", "inactive_hr_max"]
hr_df

In [ ]:
from operator    import itemgetter
from collections import namedtuple

def smooth(time, data, alpha=1, today=None):
    """Perform exponential smoothing with factor `alpha`.

    Time period is a day.
    Each time period the value of `hr` drops `alpha` times.
    The most recent data is the most valuable one.
    """
    assert 0 < alpha <= 1

    if alpha == 1: # no smoothing
        return sum(data) / len(data)

    if today is None:
        today = max(time)

    numerator = sum(alpha**((today - date).days) * hr for date, hr in zip(time, data) if not (date > today))
    denom = sum(alpha**((today - date).days) for date, hr in zip(time, data) if not (date > today))
        
    return numerator / denom

In [ ]:
fig = plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')

alpha = 0.95

color = ['r', 'g', 'b']

columns = ["hr_avg", "rhr_avg", "inactive_hr_avg"]

for col in columns:
    _df = hr_df[["Date", col, col.replace("avg", "min"), col.replace("avg", "max")]].dropna()
    smoothed_hr = [smooth(_df.Date,  _df[col], alpha=alpha, today=date) for date in _df.Date]
    smoothed_hr_max = [smooth(_df.Date,  _df[col.replace("avg", "min")], alpha=alpha, today=date) for date in _df.Date]
    smoothed_hr_min = [smooth(_df.Date,  _df[col.replace("avg", "max")], alpha=alpha, today=date) for date in _df.Date]

    plt.plot(list(_df.Date), smoothed_hr, 'o', ms=3.0, label=col)
    plt.fill_between(list(_df.Date), smoothed_hr_min, smoothed_hr_max, alpha=0.2)

plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)

plt.legend()

ax = plt.gca()
legend = ax.legend(frameon=True)
ax.legend(markerscale=2)

years = plt.matplotlib.dates.YearLocator()
months = plt.matplotlib.dates.MonthLocator()
# yearsFmt = plt.matplotlib.dates.DateFormatter('%Y')
ax.xaxis.set_major_locator(years)
ax.xaxis.set_minor_locator(months)
# ax.xaxis.set_major_formatter(yearsFmt)

plt.show()

In [ ]:
# All HR data
# db_params = ConfigManager.get_db_params()
# mon_db = MonitoringDb(db_params, False)
# start_ts = datetime.datetime.combine(datetime.date(year=2021, month=1, day=1), datetime.datetime.min.time())
# end_ts = datetime.datetime.combine(datetime.date.today(), datetime.datetime.max.time())
# hr_data_full = MonitoringHeartRate.get_for_period(mon_db, start_ts, end_ts, MonitoringHeartRate)
# hr_data_full

In [ ]:
# All Monioring data
# from garmindb.garmindb import MonitoringInfo, MonitoringIntensity, Monitoring

# db_params = ConfigManager.get_db_params()
# mon_db = MonitoringDb(db_params, False)
# start_ts = datetime.datetime.combine(datetime.date(year=2021, month=1, day=1), datetime.datetime.min.time())
# end_ts = datetime.datetime.combine(datetime.date.today(), datetime.datetime.max.time())
# mon_data_full = Monitoring.get_for_period(mon_db, start_ts, end_ts, Monitoring)
# mon_data_full